# Data process

This notebook is used to process data if needed

In [362]:
import numpy as np
import pandas as pd

In [363]:
scores = pd.read_csv('data/scores.csv')
scores.head()

,number,days,gender,age,afftype,melanch,inpatient,edu,marriage,work,madrs1,madrs2
0,condition_1,11,2,35-39,2.0,2.0,2.0,6-10,1.0,2.0,19.0,19.0
1,condition_2,18,2,40-44,1.0,2.0,2.0,6-10,2.0,2.0,24.0,11.0
2,condition_3,13,1,45-49,2.0,2.0,2.0,6-10,2.0,2.0,24.0,25.0
3,condition_4,13,2,25-29,2.0,2.0,2.0,11-15,1.0,1.0,20.0,16.0
4,condition_5,13,2,50-54,2.0,2.0,2.0,11-15,2.0,2.0,26.0,26.0


In [364]:
scores.dtypes

number        object
days           int64
gender         int64
age           object
afftype      float64
melanch      float64
inpatient    float64
edu           object
marriage     float64
work         float64
madrs1       float64
madrs2       float64
dtype: object

In [365]:
scores.describe()

,days,gender,afftype,melanch,inpatient,marriage,work,madrs1,madrs2
count,55.000000,55.000000,23.000000,20.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,12.600000,1.454545,1.739130,1.950000,1.782609,1.521739,1.869565,22.739130,20.000000
std,2.491467,0.502519,0.540824,0.223607,0.421741,0.510754,0.344350,4.797892,4.729021
min,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13.000000,11.000000
25%,13.000000,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,18.500000,16.000000
50%,13.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,24.000000,21.000000
75%,13.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,26.000000,24.500000
max,20.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,29.000000,28.000000


In [366]:
scores.isnull().sum()

number        0
days          0
gender        0
age           0
afftype      32
melanch      35
inpatient    32
edu           2
marriage     32
work         32
madrs1       32
madrs2       32
dtype: int64

There are a lot of null data in columns afftype, melanch, inpatient, marriage, work, madrs1, and madrs2. However, a quick inspection of the data reveals that these values belong to the patients in the control group, i.e., the ones that does not have depression. Thus, it is sensible to replace those value with zeros. But another observation can be made. The melanch column actually has 35 null values. This indicate there are 3 "real" null columns. We need to drop them using the exact index. 

In [367]:
scores = scores.drop([6, 7, 8])

Now, let's replace all null values in the depression related columns with zeros

In [368]:
scores.loc[:, ['afftype', 'melanch', 'inpatient', 'madrs1', 'madrs2']] = scores.loc[:, ['afftype', 'melanch', 'inpatient', 'madrs1', 'madrs2']] .fillna(0)
scores.isna().sum()

number        0
days          0
gender        0
age           0
afftype       0
melanch       0
inpatient     0
edu           2
marriage     32
work         32
madrs1        0
madrs2        0
dtype: int64

For columns marriage and work, they also have all na values for the controlled group. However it would not be sensible to replace these values with any numbers. For that reason, these columns will be leaved at they are. Finally, the column education might not be used, so it can also be leaved like that.

Now, let move on to the individual dataset. One way to go around this is to either:

* For each dataset, we summarize the data by taking the average data for each day
* We can ignore it when doing the analysis and only use them for individual anaylis. 

Either way, we need to do individual analysis first. So we can tackle the second point first. To guaranttee fairness. We can select a number from the remaining patients (i.e., from 1 to 23 excluding 7, 8, 9) in case of condition and from 1 to 32 in case of control, and select the according individual dataset.

In [369]:
conditions = np.delete(np.arange(1, 24), [6, 7, 8])
controls = np.arange(1, 33)

condition_sample = np.random.choice(conditions, 1)
control_sample = np.random.choice(controls, 1)

# CONDITION_SAMPLE = [5], [3]
# CONTROL_SAMPLE = [10], [27]

In [370]:
condition_df = pd.read_csv('data/condition/condition_{}.csv'.format(str(condition_sample[0])))
control_df = pd.read_csv('data/control/control_{}.csv'.format(str(control_sample[0])))

Now, let's do some first summarization of the dataframes

In [371]:
print(condition_df.dtypes)
print('\n')
print(control_df.dtypes)

timestamp    object
date         object
activity      int64
dtype: object


timestamp    object
date         object
activity      int64
dtype: object


In [372]:
print(condition_df.describe())
print('\n')
print(control_df.describe())

           activity
count  21648.000000
mean     265.299704
std      399.881083
min        3.000000
25%        5.000000
50%       50.000000
75%      394.000000
max     3847.000000


           activity
count  20490.000000
mean     314.047194
std      431.451563
min        0.000000
25%        0.000000
50%      130.000000
75%      469.000000
max     3869.000000


In [373]:
print(condition_df.isnull().sum())
print('\n')
print(control_df.isnull().sum())

timestamp    0
date         0
activity     0
dtype: int64


timestamp    0
date         0
activity     0
dtype: int64


There are no NaN value, so that's good. Lets get to the analyzing.

In [374]:
condition_df[['timestamp', 'date']] = condition_df[['timestamp', 'date']].apply(pd.to_datetime)
control_df[['date']] = control_df[['timestamp', 'date']].apply(pd.to_datetime)

condition_df['timestamp'] = condition_df['timestamp'].dt.hour
control_df['timestamp'] = control_df['timestamp'].dt.hour

In [375]:
condition_df.to_csv('cleaned_data/condition.csv', index=False)
control_df.to_csv('cleaned_data/control.csv', index=False)
scores.to_csv('cleaned_data/scores.csv', index=False)